In [497]:
import pandas as pd
import seaborn as sns
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
from Bio import SeqIO, Phylo

# Collate Data from Bactopia and Poppunk Analyses

In [498]:
sample_sheet = pd.read_csv('bactopia_sample_sheet.csv', sep='\t')
metadata = pd.read_csv('gas_metadata.csv', sep='\t').set_index("GAS Isolate")

In [499]:
# Parse general tool outputs

mlst = pd.concat([pd.read_csv(mlst_res, sep='\t', names=['Sample', 'Scheme', "MLST", "Allele1", "Allele2", "Allele3", "Allele4", "Allele5", "Allele6", "Allele7"]) \
                  for mlst_res in Path("bactopia").glob('*/tools/mlst/*.tsv')])
mlst['Sample'] = mlst['Sample'].str.replace('.fna.gz', '')
mlst = mlst.set_index("Sample")

emmtyper = pd.concat([pd.read_csv(emm_res, sep='\t', names=['Sample', 'Number of Clusters', 'Predicted emm-type', 'Possible emm-like alleles', 'EMM cluster']) \
                      for emm_res in Path("bactopia").glob('*/tools/emmtyper/*.tsv')])
emmtyper['Sample'] = emmtyper['Sample'].str.replace('.tmp', '')
emmtyper = emmtyper.set_index("Sample")

snptyper = pd.read_csv('assembly_snptyper/output.txt', sep='\t')
snptyper['Sample'] = snptyper['sample'].str.replace('.fna', '')
snptyper = snptyper.drop('sample', axis=1).set_index("Sample")

poppunk = pd.read_csv('poppunk/output/output_clusters.csv').set_index('Taxon')

/tmp/ipykernel_1151858/1434712552.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  mlst['Sample'] = mlst['Sample'].str.replace('.fna.gz', '')
/tmp/ipykernel_1151858/1434712552.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  emmtyper['Sample'] = emmtyper['Sample'].str.replace('.tmp', '')
/tmp/ipykernel_1151858/1434712552.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  snptyper['Sample'] = snptyper['sample'].str.replace('.fna', '')


In [500]:
# Parse virulence factor analysis outputs
virulence = pd.concat([pd.read_excel('virulence_adhesion_factors/Group A Strep Virulence Factor List for WGS.xlsx'), 
                       pd.read_excel('virulence_adhesion_factors/Additional GAS genes.xlsx')])

all_vfs = virulence['Gene Name'].unique()
all_ids = metadata.index
with open('virulence_adhesion_factors/virulence_factors.fas', 'w') as fh:
    for _, row in virulence[['Gene Name', 'DNA Sequence']].iterrows():
        fh.write(f">{row['Gene Name']}\n{row['DNA Sequence']}\n")

vf_hits = []
for out6 in Path('virulence_adhesion_factors/vf_hits/').glob('*.out6'):
    genome = out6.name.replace('.out6', '')
    out6 = pd.read_csv(out6, names="qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp qcovus".split(), sep='\t')
    out6['GAS Isolate'] = genome
    vf_hits.append(out6)
vf_hits = pd.concat(vf_hits)
vf_hits = vf_hits[(vf_hits['pident'] > 80) & (vf_hits['qcovus'] > 80)]
vf_hits_presence_absence_matrix = pd.crosstab(vf_hits['GAS Isolate'], vf_hits['saccver'])
vf_hits_presence_absence_matrix[vf_hits_presence_absence_matrix > 1] = 1

In [501]:
# Parse CovRS deletions

CovR_del = set()
CovS_del = set()

with open('virulence_adhesion_factors/covR_covS_alignment/covR_aligned.fas') as fh:
    for seq in SeqIO.parse(fh, 'fasta'):
        if seq.seq.startswith('---') or seq.seq.endswith('---'):
            CovR_del.add(seq.id.rsplit('_', 1)[0])


with open('virulence_adhesion_factors/covR_covS_alignment/covS_aligned.fas') as fh:
    for seq in SeqIO.parse(fh, 'fasta'):
        if seq.seq.startswith('---') or seq.seq.endswith('---'):
            CovS_del.add(seq.id.rsplit('_', 1)[0])
         
covRS_deletions = pd.DataFrame(index=metadata.index)
covRS_deletions['CovR Deletion'] = [1 if x in CovR_del else 0 for x in covRS_deletions.index]
covRS_deletions['CovS Deletion'] = [1 if x in CovS_del else 0 for x in covRS_deletions.index]

In [502]:
# Combine metadata and results into 1 dataframe

metadata['emm-type'] = emmtyper['Predicted emm-type']
metadata['emm-group'] = metadata['emm-type'].str.split('.').str.get(0)
metadata['MLST'] = mlst['MLST']
metadata['Poppunk Cluster'] = poppunk['Cluster']
metadata['M1UK SNPs Detected (X/27)'] = snptyper['matching_variants']
metadata['CovR Deletion'] = covRS_deletions['CovR Deletion']
metadata['CovS Deletion'] = covRS_deletions['CovS Deletion']
metadata = pd.concat([metadata, vf_hits_presence_absence_matrix], axis=1)
metadata.to_csv('gas_metadata_and_analysis_results.csv', sep='\t')

# perform emm12 only core genome phylogeny
with open('pangenome/emm12_taxa.txt', 'w') as fh:
    for x in metadata[metadata['emm-group'] == 'EMM12'].index:
        fh.write(x + '\n')

## Query metadata for clusters


In [559]:
ge_3_clusters = metadata[metadata['Poppunk Cluster'].isin(['1', '2', '16', '10', '4'])].groupby('Poppunk Cluster')
ge_3_cluster_sizes = ge_3_clusters.size()
perc_invasive = ge_3_clusters['Location'].value_counts() / ge_3_cluster_sizes * 100
perc_invasive.reset_index().query("`Location` == 'Invasive'")[0].sort_values()

1     10.869565
3     12.500000
8     16.666667
6     41.666667
4    100.000000
Name: 0, dtype: float64

In [454]:
metadata.groupby('Poppunk Cluster')['Location'].value_counts()

Poppunk Cluster  Location    
1                Non-Invasive    82
                 Invasive        10
10               Non-Invasive     7
                 Invasive         1
11               Non-Invasive     2
                 Invasive         1
127              Invasive         1
15               Non-Invasive     1
159              Invasive         1
16               Invasive         9
17               Non-Invasive     2
2                Non-Invasive     7
                 Invasive         5
24               Invasive         2
27               Invasive         2
31               Non-Invasive     1
32               Non-Invasive     1
33               Non-Invasive     2
                 Invasive         1
37               Invasive         1
4                Non-Invasive     5
                 Invasive         1
44               Non-Invasive     1
45               Non-Invasive     3
5                Non-Invasive     1
6                Non-Invasive     1
7                Invasive         

In [416]:
def percentage_of_group_invasive(metadata, cluster):
    group = metadata.groupby('Location')[cluster].value_counts().reset_index(level=0)
    group['Total in Cluster'] = metadata[cluster].value_counts()
    group[cluster + " %"] = group[cluster] / group['Total in Cluster'] * 100
    return group.sort_values(['Location', cluster + " %"], ascending=[True, False])

In [417]:
percentage_of_group_invasive(metadata, 'emm-group')

,Location,emm-group,Total in Cluster,emm-group %
emm-group,,,,
EMM49,Invasive,9,9,100.000000
EMM83,Invasive,2,2,100.000000
EMM25,Invasive,1,1,100.000000
EMM53,Invasive,1,1,100.000000
EMM6,Invasive,1,1,100.000000
EMM66,Invasive,1,1,100.000000
EMM80,Invasive,1,1,100.000000
EMM92,Invasive,1,1,100.000000
EMM77,Invasive,1,2,50.000000


## Pyseer Analyses

4 pyseer analyses were performed:
- gene presence and absence associated with invasiveness `pyseer/gene_presence_absence/output/gene_presence_absence_patterns_filtered.txt`
- gene cluster presence and absence associated with invasiveness `pyseer/gene_presence_absence/output/struct_presence_absence_patterns_filtered.txt`
- unitig presence and absence associated with invasiveness `pyseer/unitigs/unitig_results_filtered.txt`
- unitig presence and absence associated with invasiveness and controlled for by lineage `pyseer/unitigs/lineage_unitig_results_filtered.txt`
 

In [561]:
pangenome = pd.read_csv('pangenome/bactopia-runs/SHL_igas-20240616-024658/panaroo/gene_presence_absence.csv')

gene_pres_abs = pd.read_csv('pyseer/gene_presence_absence/gene_presence_absence_patterns_filtered.txt', sep='\t')
gene_pres_abs = pangenome[pangenome['Gene'].isin(gene_pres_abs['variant'])]
gene_pres_abs = gene_pres_abs.drop(['Non-unique Gene name', 'Annotation'], axis=1).set_index('Gene').T
gene_pres_abs = gene_pres_abs.notnull().astype('int')
gene_pres_abs = gene_pres_abs.fillna(0)
gene_pres_abs['Location'] = metadata.loc[gene_pres_abs.index, 'Location']
gene_pres_abs['emm-type'] = metadata.loc[gene_pres_abs.index, 'emm-type']
gene_pres_abs['Poppunk Cluster'] = metadata.loc[gene_pres_abs.index, 'Poppunk Cluster']

print(gene_pres_abs[gene_pres_abs['group_2020'] == 1])
print(gene_pres_abs.groupby('group_2020')['Location'].value_counts())

Gene               group_2020  Location emm-type Poppunk Cluster
23SG_024M0143_S4            1  Invasive  EMM12.7               1
23SH_077M0027_S17           1  Invasive   EMM6.4               8
23SH_103M0533_S19           1  Invasive  EMM12.7               1
23SH_113M0067_S13           1  Invasive  EMM80.0             159
group_2020  Location    
0           Non-Invasive    117
            Invasive         34
1           Invasive          4
Name: Location, dtype: int64


In [562]:
gene_pres_abs[gene_pres_abs['group_2020'] == 1]

Gene,group_2020,Location,emm-type,Poppunk Cluster
23SG_024M0143_S4,1,Invasive,EMM12.7,1
23SH_077M0027_S17,1,Invasive,EMM6.4,8
23SH_103M0533_S19,1,Invasive,EMM12.7,1
23SH_113M0067_S13,1,Invasive,EMM80.0,159


In [563]:
pangenome_struct = pd.read_csv('pangenome/bactopia-runs/SHL_igas-20240616-024658/panaroo/struct_presence_absence.Rtab', sep='\t')
struct_pres_abs = pd.read_csv('pyseer/gene_presence_absence/struct_presence_absence_patterns_filtered.txt', sep='\t')
struct_pres_abs = pangenome_struct[pangenome_struct['Gene'].isin(struct_pres_abs['variant'])]

struct_pres_abs = struct_pres_abs.T
struct_pres_abs.columns = struct_pres_abs.loc['Gene']
struct_pres_abs = struct_pres_abs.drop('Gene')
struct_pres_abs['Location'] = metadata.loc[struct_pres_abs.index, 'Location']
struct_pres_abs['emm-type'] = metadata.loc[struct_pres_abs.index, 'emm-type']
struct_pres_abs['Poppunk Cluster'] = metadata.loc[struct_pres_abs.index, 'Poppunk Cluster']

In [589]:
group2.groupby('Location')['Poppunk Cluster'].value_counts()

Location      Poppunk Cluster
Invasive      1                   5
              10                  1
              37                  1
Non-Invasive  1                  80
              10                  7
              17                  2
              31                  1
              6                   1
Name: Poppunk Cluster, dtype: int64

In [588]:
group2['Location'].value_counts()

Non-Invasive    91
Invasive         7
Name: Location, dtype: int64

In [565]:
group1 = struct_pres_abs.drop('group_664-group_1097-group_1161', axis=1).query('`group_148-group_664-group_1097` == 1')
group1.query("`Location` == 'Invasive'")['emm-type'].str.split('.').str.get(0).value_counts()

EMM1     5
EMM12    5
EMM49    2
EMM92    1
EMM81    1
EMM28    1
EMM53    1
EMM89    1
EMM77    1
EMM80    1
Name: emm-type, dtype: int64

In [566]:
group1 = struct_pres_abs.drop('group_664-group_1097-group_1161', axis=1).query('`group_148-group_664-group_1097` == 1')
group1.query("`Location` == 'Non-Invasive'")['emm-type'].str.split('.').str.get(0).value_counts()

EMM12    80
EMM28     7
EMM1      7
EMM89     5
EMM58     2
EMM81     2
EMM9      1
EMM11     1
EMM75     1
Name: emm-type, dtype: int64

In [567]:
group2 = struct_pres_abs.drop('group_148-group_664-group_1097', axis=1).query('`group_664-group_1097-group_1161` == 1')
group2.query("`Location` == 'Invasive'")['emm-type'].str.split('.').str.get(0).value_counts()

EMM12    5
EMM92    1
EMM28    1
Name: emm-type, dtype: int64

In [425]:
group2 = struct_pres_abs.drop('group_148-group_664-group_1097', axis=1).query('`group_664-group_1097-group_1161` == 1')
group2.query("`Location` == 'Non-Invasive'")['emm-type'].str.split('.').str.get(0).value_counts()

EMM12    80
EMM28     7
EMM58     2
EMM11     1
EMM75     1
Name: emm-type, dtype: int64

In [426]:
print(struct_pres_abs.drop('group_664-group_1097-group_1161', axis=1).query('`group_148-group_664-group_1097` == 1')['Location'].value_counts())
print(struct_pres_abs.drop('group_148-group_664-group_1097', axis=1).query('`group_664-group_1097-group_1161` == 1')['Location'].value_counts())

#print(struct_pres_abs[struct_pres_abs['group_148-group_664-group_1097'] == 1].drop('g'))

Non-Invasive    106
Invasive         19
Name: Location, dtype: int64
Non-Invasive    91
Invasive         7
Name: Location, dtype: int64


In [428]:
print(struct_pres_abs.drop('group_664-group_1097-group_1161', axis=1).groupby('group_148-group_664-group_1097')['Location'].value_counts())
print(struct_pres_abs.drop('group_664-group_1097-group_1161', axis=1).query('Location == "Invasive"').groupby('group_148-group_664-group_1097')['emm-type'].value_counts())

group_148-group_664-group_1097  Location    
0                               Invasive         19
                                Non-Invasive     11
1                               Non-Invasive    106
                                Invasive         19
Name: Location, dtype: int64
group_148-group_664-group_1097  emm-type
0                               EMM49.0     7
                                EMM12.37    2
                                EMM83.1     2
                                EMM12.0     1
                                EMM12.7     1
                                EMM12.8     1
                                EMM22.0     1
                                EMM25.4     1
                                EMM6.4      1
                                EMM66.0     1
                                EMM82.0     1
1                               EMM1.0      5
                                EMM12.0     3
                                EMM49.0     2
                                E

In [249]:
pangenome_bin = pangenome.drop(['Non-unique Gene name', 'Annotation'], axis=1).set_index("Gene").notnull().astype('int').fillna(0).T

In [253]:
pangenome_bin['Location'] = metadata.loc[pangenome_bin.index, 'Location']
pangenome_bin

Gene,rpmJ,group_2190,rpmH,rpmG3,group_2144,group_2141,group_2137,group_2120,group_2111,rpmD,...,cyaB,group_32,group_23,group_18,group_12,group_9,group_5,group_1,group_0,Location
23SB_106M0036_S15,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Invasive
23SB_143M0036_S5,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Invasive
23SC_014M0062_S3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Invasive
23SC_035M0015_S9,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Invasive
23SC_083M0072_S18,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Invasive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R93_S29,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Non-Invasive
R94_S30,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Non-Invasive
R95_S31,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Non-Invasive
R96_S32,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Non-Invasive


## Data in Manuscript
### Abstract
Number of genomes

In [307]:
metadata.shape[0]

155

Number of invasive and non-invasive genomes

In [308]:
metadata['Location'].value_counts()

Non-Invasive    117
Invasive         38
Name: Location, dtype: int64

Top-level EMM% for non-invasive

In [309]:
metadata.loc[metadata['Location'] == 'Non-Invasive', 'emm-type'].str.split('.').str.get(0).value_counts() / metadata['Location'].value_counts().loc['Non-Invasive'] * 100

EMM12    70.085470
EMM28     5.982906
EMM1      5.982906
EMM89     4.273504
EMM2      2.564103
EMM22     1.709402
EMM58     1.709402
EMM81     1.709402
EMM82     0.854701
EMM4      0.854701
EMM87     0.854701
EMM9      0.854701
EMM11     0.854701
EMM77     0.854701
EMM75     0.854701
Name: emm-type, dtype: float64

Top-level EMM% for invasive

In [310]:
metadata.loc[metadata['Location'] == 'Invasive', 'emm-type'].str.split('.').str.get(0).value_counts() / metadata['Location'].value_counts().loc['Invasive'] * 100

EMM12    26.315789
EMM49    23.684211
EMM1     13.157895
EMM83     5.263158
EMM66     2.631579
EMM25     2.631579
EMM22     2.631579
EMM92     2.631579
EMM81     2.631579
EMM28     2.631579
EMM82     2.631579
EMM53     2.631579
EMM89     2.631579
EMM6      2.631579
EMM77     2.631579
EMM80     2.631579
Name: emm-type, dtype: float64

% of EMM1.0 isolates with M1UK SNP set

In [311]:
total = metadata.loc[(metadata['emm-type'].str.startswith('EMM1.')), 'M1UK SNPs Detected (X/27)'].value_counts().sum()
metadata.loc[(metadata['emm-type'].str.startswith('EMM1.')), 'M1UK SNPs Detected (X/27)'].value_counts() / total * 100

27    58.333333
0     41.666667
Name: M1UK SNPs Detected (X/27), dtype: float64

Invasive/non-invasive by CovS deletion

In [312]:
metadata[metadata['CovS Deletion'] == 1]['Location'].value_counts()

Invasive    7
Name: Location, dtype: int64

### Introduction

In [313]:
metadata['Location'].value_counts()

Non-Invasive    117
Invasive         38
Name: Location, dtype: int64

In [314]:
metadata['Location'].value_counts().sum()

155

In [315]:
print(f"{metadata['Date Frozen'].min()} - {metadata['Date Frozen'].max()}")

2023-01-07 - 2023-05-31


In [316]:
!grep -c "^>" virulence_adhesion_factors/virulence_factors.fas

63
